In [1]:
import os 
import pickle
import torch
import rootutils
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
rootutils.setup_root('/idiap/temp/pvuillecard/projects/gaze_pretrain/notebooks', indicator=".project-root", pythonpath=True)
# plot the results 
import matplotlib.pyplot as plt
from gaze_module.utils.metrics import AngularError
from gaze_module.utils.metrics_utils import spherical2cartesial

In [2]:
path_exp = '/idiap/temp/pvuillecard/projects/gaze_pretrain/logs/experiments/scripts/generalize.yaml/run_2024-06-26_21-19-23/logs/train/runs/run_0/metric/test/prediction_test_gaze_epoch_0.pkl'

with open(path_exp, 'rb') as f:
    exp_results = pickle.load(f)

angular_error_all = {}
mode_angular = 'cartesian'
k =3
exp_results['frame_pred'] = exp_results['frame_pred'].view(-1, k).numpy()
exp_results['frame_gt'] = exp_results['frame_gt'].view(-1, k).numpy()
exp_results['frame_id'] = exp_results['frame_id'].view(-1).numpy()
exp_results['video_id'] = exp_results['video_id'].view(-1).numpy()
exp_results['person_id'] = exp_results['person_id'].view(-1).numpy()
exp_results['data_id'] = exp_results['data_id'].view(-1).numpy()


with open('/idiap/temp/pvuillecard/projects/gaze_pretrain/logs/experiments/gaze360_model/run_2024-06-19_11-13-54/logs/train/runs/run_0/prediction/pred_gaze_gazefollow.pkl', 'rb') as f:
    image_db_follow = pickle.load(f)

for k in image_db_follow.keys():
    gaze_dir = image_db_follow[k]['gaze_vector_pred']
    # compute the angular error with center (0,0,-1)
    angular_error = 180/np.pi*np.arccos(np.dot(gaze_dir, np.array([0, 0, -1])) / (np.linalg.norm(gaze_dir) * np.linalg.norm(np.array([0, 0, -1]))))
    image_db_follow[k]['angular_error'] = angular_error


#################################################################################################
# GazeFollow TESTING
#################################################################################################
pred_follow = {}
for i in range(len(exp_results['frame_id'])):
    if exp_results['data_id'][i] == 4:
        pred_follow[f'frame_{exp_results["frame_id"][i]:08d}_face_{exp_results["person_id"][i]:08d}'] = {
            'frame_pred': exp_results['frame_pred'][i],
            'frame_gt': exp_results['frame_gt'][i]
        }
angular_error_full = AngularError(mode = mode_angular)
angular_error_180 = AngularError(mode = mode_angular)
angular_error_20 = AngularError(mode = mode_angular)
angular_error_2D = AngularError(mode = mode_angular)

if len(pred_follow) > 0:
    for k in pred_follow.keys():
        
        print(torch.from_numpy(pred_follow[k]['frame_gt']).unsqueeze(0)[:,:2].shape)
        angular_error_2D.update(torch.from_numpy(pred_follow[k]['frame_gt']).unsqueeze(0)[:,:2],
                            torch.from_numpy(pred_follow[k]['frame_pred']).unsqueeze(0)[:,:2])
        input = torch.from_numpy(pred_follow[k]['frame_gt']).unsqueeze(0)[:,:2]
        target = torch.from_numpy(pred_follow[k]['frame_pred']).unsqueeze(0)[:,:2]
        input_cart = torch.nn.functional.normalize(input, p=2, dim=1)
        target_cart = torch.nn.functional.normalize(target, p=2, dim=1)

        sim = torch.nn.functional.cosine_similarity(input_cart, target_cart, dim=1, eps=1e-10)
        print(sim)
        #sim = torch.nn.functional.hardtanh_(sim, min_val=-1+1e-8, max_val=1-1e-8)
        acos_sim = torch.acos(sim)
        acos_sim[sim >= (1.0 - 1e-8)] = 0
        acos_sim[sim <= (-1.0 + 1e-8)] = math.pi
        print(acos_sim)
        output_dot = acos_sim.sum()*180/math.pi
       
        # raise error if the output is not a number
        if torch.isnan(output_dot):
            raise ValueError("The output is not a number")
        
        
        angular_error_full.update(torch.from_numpy(pred_follow[k]['frame_gt']).unsqueeze(0),
                            torch.from_numpy(pred_follow[k]['frame_pred']).unsqueeze(0))
        if image_db_follow[k]['angular_error'] <= 90:
            angular_error_180.update(torch.from_numpy(pred_follow[k]['frame_gt']).unsqueeze(0),
                                        torch.from_numpy(pred_follow[k]['frame_pred']).unsqueeze(0))
        if image_db_follow[k]['angular_error'] <= 20:
            angular_error_20.update(torch.from_numpy(pred_follow[k]['frame_gt']).unsqueeze(0),
                                        torch.from_numpy(pred_follow[k]['frame_pred']).unsqueeze(0))
        
    angular_error_all["GazeFollow_full"] = angular_error_full.compute()
    angular_error_all["GazeFollow_180"] = angular_error_180.compute()
    angular_error_all["GazeFollow_20"] = angular_error_20.compute()
    angular_error_all["GazeFollow_2D"] = angular_error_2D.compute()
 

torch.Size([1, 2])
tensor([0.9964])
tensor([0.0848])
torch.Size([1, 2])
tensor([0.7342])
tensor([0.7464])
torch.Size([1, 2])
tensor([0.9907])
tensor([0.1362])
torch.Size([1, 2])
tensor([0.9267])
tensor([0.3852])
torch.Size([1, 2])
tensor([-0.9899])
tensor([2.9994])
torch.Size([1, 2])
tensor([0.9680])
tensor([0.2536])
torch.Size([1, 2])
tensor([0.9313])
tensor([0.3727])
torch.Size([1, 2])
tensor([0.9173])
tensor([0.4095])
torch.Size([1, 2])
tensor([0.9997])
tensor([0.0239])
torch.Size([1, 2])
tensor([0.2007])
tensor([1.3687])
torch.Size([1, 2])
tensor([0.9711])
tensor([0.2411])
torch.Size([1, 2])
tensor([0.8936])
tensor([0.4654])
torch.Size([1, 2])
tensor([0.9993])
tensor([0.0369])
torch.Size([1, 2])
tensor([0.9293])
tensor([0.3782])
torch.Size([1, 2])
tensor([0.8728])
tensor([0.5100])
torch.Size([1, 2])
tensor([0.9876])
tensor([0.1578])
torch.Size([1, 2])
tensor([-0.3955])
tensor([1.9774])
torch.Size([1, 2])
tensor([-0.0418])
tensor([1.6127])
torch.Size([1, 2])
tensor([0.1368])
tensor(

In [3]:
angular_error_all

{'GazeFollow_full': tensor(35.3124),
 'GazeFollow_180': tensor(29.5362),
 'GazeFollow_20': tensor(25.4553),
 'GazeFollow_2D': tensor(39.5844)}

In [10]:
torch.arccos(torch.tensor(1.0 - 1e-8))

tensor(0.)